In [20]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import prettytensor as pt

In [3]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [71]:
result = unpickle('cifar-10-batches-py/data_batch_{}'.format(2))

In [224]:
def convert_image(image):#input image with size 1*3072 output image that fits shape ready to feed in tensorflow
    test_image = image.reshape([3,1024])
    layer1 = test_image[0].reshape((32,32,1))
    layer2 = test_image[1].reshape((32,32,1))
    layer3 = test_image[2].reshape((32,32,1))
    c=np.concatenate((layer1,layer2,layer3), axis=2)
    c = c.reshape((1,32,32,3))
    return c
def convert_group_image(image_group):
    a = []
    size = np.shape(image_group)[0]
    for i in range(size):
        image = image_group[i]
        test_image = image.reshape([3,1024])
        layer1 = test_image[0].reshape((32,32,1))
        layer2 = test_image[1].reshape((32,32,1))
        layer3 = test_image[2].reshape((32,32,1))
        c=np.concatenate((layer1,layer2,layer3), axis=2)
        a.append(c)
    return a

def convert2dummy(fine_labels):
    dummy_var = []
    zeros = np.zeros(10)

    for item in fine_labels:
        zeros = np.zeros(10)
        zeros[item] = 1
        dummy_var.append(zeros)
    return dummy_var 

def show_results(image,result_y):
    items = ['airplane','automobile','bird','cat','deer','dot','frog','horse','ship','truck']
    a,b = plt.subplots(3,3)
    a.subplots_adjust(hspace=0.3, wspace=0.3)
    for index,image_place in enumerate(b.flat):
        image_place.imshow(image[index])
        image_place.set_xlabel(items[np.argmax(result_y[index])])
        image_place.set_xticks([])
        image_place.set_yticks([])

In [72]:
b = 10 #batch size

In [160]:
#labels
#filenames
#data
#batch_label
image_set = result[b'data']
image_set = convert_group_image(image_set)
y_labels = result[b'labels']
y_true = convert2dummy(y_labels)
np.shape(y_true)

(10000, 10)

In [154]:
X = tf.placeholder(shape=(b,32,32,3),dtype=tf.float32)
Y = tf.placeholder(shape=(b,10),dtype=tf.float32)

In [155]:
W1 = tf.Variable(tf.truncated_normal(shape=(3,3,3,72), mean=0, stddev=1))
B1 = tf.Variable(tf.constant(0,shape=[72],dtype=tf.float32))

W2 = tf.Variable(tf.truncated_normal(shape=(3,3,72,144), mean=0, stddev=1))
B2 = tf.Variable(tf.constant(0,shape=[144],dtype=tf.float32))

W3 = tf.Variable(tf.truncated_normal(shape=(3,3,144,288), mean=0, stddev=1))
B3 = tf.Variable(tf.constant(0,shape=[288],dtype=tf.float32))

fc_W1 = tf.Variable(tf.truncated_normal(shape=(4608,256), mean=0, stddev=1))
fc_B1 = tf.Variable(tf.constant(0,shape=[10,256],dtype=tf.float32))

fc_W2 = tf.Variable(tf.truncated_normal(shape=(256,10), mean=0, stddev=1))
fc_B2 = tf.Variable(tf.constant(0,shape=[10,10],dtype=tf.float32))

In [156]:
l1_1 = tf.nn.conv2d(input=X,filter=W1,strides=[1,1,1,1],padding='SAME')
l1_1 = tf.nn.bias_add(l1_1,B1)
l1_2 = tf.nn.relu(l1_1)
l1_3 = tf.nn.max_pool(l1_2,[1,2,2,1],[1,2,2,1],padding='SAME')

l2_1 = tf.nn.conv2d(input=l1_3,filter=W2,strides=[1,1,1,1],padding='SAME')
l2_1 = tf.nn.bias_add(l2_1,B2)
l2_2 = tf.nn.relu(l2_1)
l2_3 = tf.nn.max_pool(l2_2,[1,2,2,1],[1,2,2,1],padding='SAME')

l3_1 = tf.nn.conv2d(input=l2_3,filter=W3,strides=[1,1,1,1],padding='SAME')
l3_1 = tf.nn.bias_add(l3_1,B3)
l3_2 = tf.nn.relu(l3_1)
l3_3 = tf.nn.max_pool(l3_2,[1,2,2,1],[1,2,2,1],padding='SAME')

In [157]:
flat = tf.layers.flatten(l3_3)

In [158]:
fc1 = tf.matmul(flat,fc_W1)
fc1 = tf.add(fc1,fc_B1)

fc2 = tf.matmul(fc1,fc_W2)
fc2 = tf.add(fc2,fc_B2)

In [191]:
pred_y = tf.nn.softmax(fc2)

In [159]:
cost = tf.losses.softmax_cross_entropy(onehot_labels=Y,logits=fc2)

In [165]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

In [147]:
sess = tf.Session()

In [167]:
sess.run(tf.global_variables_initializer())

In [223]:
for i in range(1000):
    rand_num = np.random.randint(low=0,high=999)#needs hone here for it may not agree with images 
    sess.run(optimizer,feed_dict={X:image_set[rand_num:b+rand_num],Y:y_true[rand_num:b+rand_num]})
    output = sess.run(cost,feed_dict={X:image_set[rand_num:b+rand_num],Y:y_true[rand_num:b+rand_num]})
    if i%100 == 0:
        result_y = sess.run(pred_y,feed_dict={X:image_set[rand_num:b+rand_num],Y:y_true[rand_num:b+rand_num]})
        show_results(image_set[rand_num:b+rand_num],result_y)
        print('loss is:',output)

TypeError: unsupported operand type(s) for +: 'AxesSubplot' and 'int'

In [194]:
y_true[0:10]

[array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.]),
 array([ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.])]

In [195]:
result_y

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.]], dtype=float32)

In [130]:
sess.close()